# Cleaning Data with PySpark

Working with data is tricky - working with millions or even billions of rows is worse. Did you receive some data processing code written on a laptop with fairly pristine data? Chances are you’ve probably been put in charge of moving a basic data process from prototype to production. You may have worked with real world datasets, with missing fields, bizarre formatting, and orders of magnitude more data. Even if this is all new to you, this course helps you learn what’s needed to prepare data processes using Python with Apache Spark. You’ll learn terminology, methods, and some best practices to create a performant, maintainable, and understandable data processing platform.

## Table of Contents

- [Introduction](#intro)
- [Manipulating DataFrames in the real wold](#man)
- [Improving Performance](#perf)

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

path = "data/dc33/"

In [3]:
from pyspark import SparkContext
sc = SparkContext("local", "First App")
print(sc)

<SparkContext master=local appName=First App>


In [4]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName('First App').getOrCreate()

---
<a id='intro'></a>

## Intro to data cleaning with Apache Spark

<img src="images/spark3_001.png" alt="" style="width: 800px;"/>

<img src="images/spark3_002.png" alt="" style="width: 800px;"/>

<img src="images/spark3_003.png" alt="" style="width: 800px;"/>

<img src="images/spark3_004.png" alt="" style="width: 800px;"/>

<img src="images/spark3_005.png" alt="" style="width: 800px;"/>

## Defining a schema

Creating a defined schema helps with data quality and import performance. As mentioned during the lesson, we'll create a simple schema to read in the following columns:

- Name
- Age
- City

The Name and City columns are `StringType()` and the Age column is an `IntegerType()`.

- Import * from the pyspark.sql.types library.
- Define a new schema using the StructType method.
- Define a StructField for name, age, and city. Each field should correspond to the correct datatype and not be nullable.

In [2]:
# Import the pyspark.sql.types library
from pyspark.sql.types import *

# Define a new schema using the StructType method
people_schema = StructType([
  # Define a StructField for each field
  StructField('name', StringType(), False),
  StructField('age', IntegerType(), False),
  StructField('city', StringType(), False)
])

## Immutability and lazy processing

<img src="images/spark3_006.png" alt="" style="width: 800px;"/>

<img src="images/spark3_007.png" alt="" style="width: 800px;"/>

<img src="images/spark3_008.png" alt="" style="width: 800px;"/>

<img src="images/spark3_009.png" alt="" style="width: 800px;"/>

Spark takes advantage of data immutability to efficiently share / create new data representations throughout the cluster.

## Using lazy processing

`Lazy processing operations` will usually return in about the same amount of time regardless of the actual quantity of data. Remember that this is due to `Spark not performing any transformations until an action is requested`.

For this exercise, we'll be defining a Data Frame (aa_dfw_df) and add a couple transformations. Note the amount of time required for the transformations to complete when defined vs when the data is actually queried. These differences may be short, but they will be noticeable. When working with a full Spark cluster with larger quantities of data the difference will be more apparent.

- Load the Data Frame.
- Add the transformation for F.lower() to the Destination Airport column.
- Drop the Destination Airport column from the Data Frame aa_dfw_df. Note the time for these operations to complete.
- Show the Data Frame, noting the time difference for this action to complete.

In [10]:
from pyspark.sql import functions as F

# Load the CSV file
aa_dfw_df = spark.read.format('csv').options(Header=True).load(path+'AA_DFW_2017_Departures_Short.csv.gz')

# Add the airport column using the F.lower() method
aa_dfw_df = aa_dfw_df.withColumn('airport', F.lower(aa_dfw_df['Destination Airport']))

# Drop the Destination Airport column
aa_dfw_df = aa_dfw_df.drop(aa_dfw_df['Destination Airport'])

# Show the DataFrame
aa_dfw_df.show()

+-----------------+-------------+-----------------------------+-------+
|Date (MM/DD/YYYY)|Flight Number|Actual elapsed time (Minutes)|airport|
+-----------------+-------------+-----------------------------+-------+
|       01/01/2017|         0005|                          537|    hnl|
|       01/01/2017|         0007|                          498|    ogg|
|       01/01/2017|         0037|                          241|    sfo|
|       01/01/2017|         0043|                          134|    dtw|
|       01/01/2017|         0051|                           88|    stl|
|       01/01/2017|         0060|                          149|    mia|
|       01/01/2017|         0071|                          203|    lax|
|       01/01/2017|         0074|                           76|    mem|
|       01/01/2017|         0081|                          123|    den|
|       01/01/2017|         0089|                          161|    slc|
|       01/01/2017|         0096|                           84| 

You've just seen how lazy processing works in action. Remember when working with Spark that no transformations take effect until you apply an action. This can be confusing at times, but is one of the underpinnings of Spark's power.

## Understanding Parquet

<img src="images/spark3_010.png" alt="" style="width: 800px;"/>

<img src="images/spark3_011.png" alt="" style="width: 800px;"/>

<img src="images/spark3_012.png" alt="" style="width: 800px;"/>

<img src="images/spark3_013.png" alt="" style="width: 800px;"/>

<img src="images/spark3_014.png" alt="" style="width: 800px;"/>

## Saving a DataFrame in Parquet format

When working with Spark, you'll often start with CSV, JSON, or other data sources. This provides a lot of flexibility for the types of data to load, but it is not an optimal format for Spark. The `Parquet format` is a columnar data store, allowing Spark to use `predicate pushdown`. This means Spark will only process the data necessary to complete the operations you define versus reading the entire dataset. This gives Spark more flexibility in accessing the data and often drastically improves performance on large datasets.

In this exercise, we're going to practice creating a new Parquet file and then process some data from it.

The spark object and the df1 and df2 DataFrames have been setup for you.

- View the row count of df1 and df2.
- Combine df1 and df2 in a new DataFrame named df3 with the union method.
- Save df3 to a parquet file named AA_DFW_ALL.parquet.
- Read the AA_DFW_ALL.parquet file and show the count.

In [ ]:
# View the row count of df1 and df2
print("df1 Count: %d" % df1.count())
print("df2 Count: %d" % df2.count())

# Combine the DataFrames into one
df3 = df1.union(df2)

# Save the df3 DataFrame in Parquet format
df3.write.parquet('AA_DFW_ALL.parquet', mode='overwrite')

# Read the Parquet file into a new DataFrame and run a count
print(spark.read.parquet('AA_DFW_ALL.parquet').count())

```
<script.py> output:
    df1 Count: 139359
    df2 Count: 119911
    259270
```

## SQL and Parquet

`Parquet files are perfect as a backing data store for SQL queries in Spark`. While it is possible to run the same queries directly via Spark's Python functions, sometimes it's easier to run SQL queries alongside the Python options.

For this example, we're going to read in the Parquet file we created in the last exercise and register it as a SQL table. Once registered, we'll run a quick query against the table (aka, the Parquet file).

The spark object and the AA_DFW_ALL.parquet file are available for you automatically.

- Import the AA_DFW_ALL.parquet file into flights_df.
- Use the createOrReplaceTempView method to alias the flights table.
- Run a Spark SQL query against the flights table.

In [ ]:
# Read the Parquet file into flights_df
flights_df = spark.read.parquet('AA_DFW_ALL.parquet')

# Register the temp table
flights_df.createOrReplaceTempView('flights')

# Run a SQL query of the average flight duration
avg_duration = spark.sql('SELECT avg(flight_duration) from flights').collect()[0]
print('The average flight time is: %d' % avg_duration)

```
<script.py> output:
    The average flight time is: 151
```
You've just run a SQL query against a Parquet data source. When building production Spark code, you'll often port SQL operations directly.

---
<a id='man'></a>

## Manipulating DataFrames in the real wold

## DataFrame column operations

<img src="images/spark3_015.png" alt="" style="width: 800px;"/>

<img src="images/spark3_016.png" alt="" style="width: 800px;"/>

<img src="images/spark3_017.png" alt="" style="width: 800px;"/>

<img src="images/spark3_018.png" alt="" style="width: 800px;"/>

<img src="images/spark3_019.png" alt="" style="width: 800px;"/>

## Filtering column content with Python

You've looked at using various operations on DataFrame columns - now you can modify a real dataset. The DataFrame `voter_df` contains information regarding the voters on the Dallas City Council from the past few years. This truncated DataFrame contains the date of the vote being cast and the name and position of the voter. Your manager has asked you to clean this data so it can later be integrated into some desired reports. The primary task is to remove any null entries or odd characters and return a specific set of voters where you can validate their information.

This is often one of the first steps in data cleaning - removing anything that is obviously outside the format. For this dataset, make sure to look at the original data and see what looks out of place for the VOTER_NAME column.

The `pyspark.sql.functions` library is already imported under the alias `F`.

- Show the distinct VOTER_NAME entries.
- Filter voter_df where the VOTER_NAME is 1-20 characters in length.
- Filter out voter_df where the VOTER_NAME contains an _.
- Show the distinct VOTER_NAME entries again.

In [ ]:
# Show the distinct VOTER_NAME entries
voter_df.select('VOTER_NAME').distinct().show(40, truncate=False)

# Filter voter_df where the VOTER_NAME is 1-20 characters in length
voter_df = voter_df.filter('length(VOTER_NAME) > 0 and length(VOTER_NAME) < 20')

# Filter out voter_df where the VOTER_NAME contains an underscore
voter_df = voter_df.filter(~ F.col('VOTER_NAME').contains('_'))

# Show the distinct VOTER_NAME entries again
voter_df.select('VOTER_NAME').distinct().show(40, truncate=False)

## Modifying DataFrame columns

Previously, you filtered out any rows that didn't conform to something generally resembling a name. Now based on your earlier work, your manager has asked you to create two new columns - first_name and last_name. She asks you to split the VOTER_NAME column into words on any space character. You'll treat the last word as the last_name, and all other words as the first_name. You'll be using some new functions in this exercise including `.split()`, `.size()`, and `.getItem()`.

Please note that these operations are always somewhat specific to the use case. Having your data conform to a format often matters more than the specific details of the format. Rarely is a data cleaning task meant just for one person - matching a defined format allows for easier sharing of the data later (ie, Paul doesn't need to worry about names - Mary already cleaned the dataset).

The filtered voter DataFrame from your previous exercise is available as voter_df. The pyspark.sql.functions library is available under the alias F.

- Add a new column called splits holding the list of possible names.
- Use the getItem() method and create a new column called first_name.
- Get the last entry of the splits list and create a column called last_name.
- Drop the splits column and show the new voter_df.

In [ ]:
# Add a new column called splits separated on whitespace
voter_df = voter_df.withColumn('splits', F.split(voter_df.VOTER_NAME, '\s+'))

# Create a new column called first_name based on the first item in splits
voter_df = voter_df.withColumn('first_name', voter_df.splits.getItem(0))

# Get the last entry of the splits list and create a column called last_name
voter_df = voter_df.withColumn('last_name', voter_df.splits.getItem(F.size('splits') - 1))

# Drop the splits column
voter_df = voter_df.drop('splits')

# Show the voter_df DataFrame
voter_df.show()

This exercise requires some creative thought to determine how best to handle the VOTER_NAME column to get the desired result. You may be wondering about the middle initial. We've left it out of this exercise for clarity, but consider the various ways you could add it to a given column. The string processing functions in Spark allow a wide range of operations to suit most requirements. Always refer to the Spark documentation when you need to modify a string column: you may be surprised at the available options!

## Conditional DataFrame column operations

<img src="images/spark3_020.png" alt="" style="width: 800px;"/>

<img src="images/spark3_021.png" alt="" style="width: 800px;"/>

<img src="images/spark3_022.png" alt="" style="width: 800px;"/>

<img src="images/spark3_023.png" alt="" style="width: 800px;"/>

## when() example

The `when()` clause lets you conditionally modify a Data Frame based on its content. You'll want to modify our voter_df DataFrame to add a random number to any voting member that is defined as a "Councilmember".

The voter_df DataFrame is defined and available to you. The pyspark.sql.functions library is available as F. You can use F.rand() to generate the random value.

- Add a column to voter_df named random_val with the results of the `F.rand()` method for any voter with the title Councilmember.
- Show some of the DataFrame rows, noting whether the `.when()` clause worked.

In [ ]:
# Add a column to voter_df for any voter with the title **Councilmember**
voter_df = voter_df.withColumn('random_val',
                               when(voter_df.TITLE == 'Councilmember', F.rand()))

# Show some of the DataFrame rows, noting whether the when clause worked
voter_df.show()

## When / Otherwise

This requirement is similar to the last, but now you want to add multiple values based on the voter's position. Modify your voter_df DataFrame to add a random number to any voting member that is defined as a Councilmember. Use 2 for the Mayor and 0 for anything other position.

The voter_df Data Frame is defined and available to you. The pyspark.sql.functions library is available as F. You can use F.rand() to generate the random value.

- Add a column to voter_df named random_val with the results of the F.rand() method for any voter with the title Councilmember. Set random_val to 2 for the Mayor. Set any other title to the value 0.
- Show some of the Data Frame rows, noting whether the clauses worked.
- Use the .filter clause to find 0 in random_val.

In [ ]:
# Add a column to voter_df for a voter based on their position
voter_df = voter_df.withColumn('random_val',
                               when(voter_df.TITLE == 'Councilmember', F.rand())
                               .when(voter_df.TITLE == 'Mayor', 2)
                               .otherwise(0))

# Show some of the DataFrame rows
voter_df.show()

# Use the .filter() clause with random_val
voter_df.filter(voter_df.random_val == 0).show()

You've successfully used multiple when clauses and the otherwise clause to modify a Data Frame. When clauses can be useful for changing errant data in your Data Frames without extensive work. Make sure to consider using when / otherwise if you ever need to perform conditional steps on your data cleaning processes.

## User defined functions

<img src="images/spark3_024.png" alt="" style="width: 800px;"/>

<img src="images/spark3_025.png" alt="" style="width: 800px;"/>

<img src="images/spark3_026.png" alt="" style="width: 800px;"/>

## Using user defined functions in Spark

You've seen some of the power behind Spark's built-in string functions when it comes to manipulating DataFrames. However, once you reach a certain point, it becomes difficult to process the data in a without creating a rat's nest of function calls. Here's one place where you can use User Defined Functions to manipulate our DataFrames.

For this exercise, we'll use our voter_df DataFrame, but you're going to replace the first_name column with the first and middle names.

The pyspark.sql.functions library is available under the alias F. The classes from pyspark.sql.types are already imported.

- Edit the getFirstAndMiddle() function to return a space separated string of names, except the last entry in the names list.
- Define the function as a user-defined function. It should return a string type.
- Create a new column on voter_df called first_and_middle_name using your UDF.
- Drop the "first_name" and "splits" columns (on separate lines), then show the Data Frame.

In [ ]:
def getFirstAndMiddle(names):
  # Return a space separated string of names
  return ' '.join(names[-1])

# Define the method as a UDF
udfFirstAndMiddle = F.udf(getFirstAndMiddle, StringType())

# Create a new column using your UDF
voter_df = voter_df.withColumn('first_and_middle_name', udfFirstAndMiddle(voter_df.splits))

# Drop the unecessary columns then show the DataFrame
voter_df = voter_df.drop('first_name')
voter_df = voter_df.drop('splits')
voter_df.show()

## Partitioning and lazy processing

<img src="images/spark3_027.png" alt="" style="width: 800px;"/>

<img src="images/spark3_028.png" alt="" style="width: 800px;"/>

<img src="images/spark3_029.png" alt="" style="width: 800px;"/>

<img src="images/spark3_030.png" alt="" style="width: 800px;"/>

<img src="images/spark3_031.png" alt="" style="width: 800px;"/>

## Adding an ID Field

When working with data, you sometimes only want to access certain fields and perform various operations. In this case, find all the unique voter names from the DataFrame and add a unique ID number. Remember that `Spark IDs are assigned based on the DataFrame partition - as such the ID values may be much greater than the actual number of rows in the DataFrame`.

With Spark's lazy processing, `the IDs are not actually generated until an action is performed and can be somewhat random depending on the size of the dataset`.

The spark session and a Spark DataFrame df containing the DallasCouncilVotes.csv.gz file are available in your workspace. The pyspark.sql.functions library is available under the alias F.

- Select the unique entries from the column VOTER NAME and create a new DataFrame called voter_df.
- Count the rows in the voter_df DataFrame.
- Add a ROW_ID column using Spark's id function.
- Show the rows with the 10 highest ROW_IDs.

In [ ]:
# Select all the unique council voters
voter_df = df.select(df["VOTER NAME"]).distinct()

# Count the rows in voter_df
print("\nThere are %d rows in the voter_df DataFrame.\n" % voter_df.count())

# Add a ROW_ID
voter_df = voter_df.withColumn('ROW_ID', F.monotonically_increasing_id())

# Show the rows with 10 highest IDs in the set
voter_df.orderBy(voter_df.ROW_ID.desc()).show(10)

## IDs with different partitions

You've just completed adding an ID field to a DataFrame. Now, take a look at what happens when you do the same thing on DataFrames containing a different number of partitions.

To check the number of partitions, use the method `.rdd.getNumPartitions()` on a DataFrame.

The spark session and two DataFrames, voter_df and voter_df_single, are available in your workspace. The instructions will help you discover the difference between the DataFrames. The pyspark.sql.functions library is available under the alias F.

- Print the number of partitions on each DataFrame.
- Add a ROW_ID field to each DataFrame.
- Show the top 10 IDs in each DataFrame.

In [ ]:
# Print the number of partitions in each DataFrame
print("\nThere are %d partitions in the voter_df DataFrame.\n" % voter_df.rdd.getNumPartitions())
print("\nThere are %d partitions in the voter_df_single DataFrame.\n" % voter_df_single.rdd.getNumPartitions())

# Add a ROW_ID field to each DataFrame
voter_df = voter_df.withColumn('ROW_ID', F.monotonically_increasing_id())
voter_df_single = voter_df_single.withColumn('ROW_ID', F.monotonically_increasing_id())

# Show the top 10 IDs in each DataFrame 
voter_df.orderBy(voter_df.ROW_ID.desc()).show(10)
voter_df_single.orderBy(voter_df_single.ROW_ID.desc()).show(10)

```
<script.py> output:
    
    There are 200 partitions in the voter_df DataFrame.
    
    
    There are 1 partitions in the voter_df_single DataFrame.
    
    +--------------------+-------------+
    |          VOTER NAME|       ROW_ID|
    +--------------------+-------------+
    |        Lee Kleinman|1709396983808|
    |  the  final  201...|1700807049217|
    |         Erik Wilson|1700807049216|
    |  the  final   20...|1683627180032|
    | Carolyn King Arnold|1632087572480|
    | Rickey D.  Callahan|1597727834112|
    |   the   final  2...|1443109011456|
    |    Monica R. Alonzo|1382979469312|
    |     Lee M. Kleinman|1228360646656|
    |   Jennifer S. Gates|1194000908288|
    +--------------------+-------------+
    only showing top 10 rows
    
    +--------------------+------+
    |          VOTER NAME|ROW_ID|
    +--------------------+------+
    |        Lee Kleinman|    35|
    |  the  final  201...|    34|
    |         Erik Wilson|    33|
    |  the  final   20...|    32|
    | Carolyn King Arnold|    31|
    | Rickey D.  Callahan|    30|
    |   the   final  2...|    29|
    |    Monica R. Alonzo|    28|
    |     Lee M. Kleinman|    27|
    |   Jennifer S. Gates|    26|
    +--------------------+------+
    only showing top 10 rows
```
Notice the drastic difference in the 'ROW_ID' values between the two Data Frames. Understanding how lazy processing and partitioning behave are integral to mastering Spark. Make sure to always test your assumptions when creating a Spark workflow to avoid nasty suprises in production.

## More ID tricks

Once you define a Spark process, you'll likely want to use it many times. Depending on your needs, you may want to start your IDs at a certain value so there isn't overlap with previous runs of the Spark task. This behavior is similar to how IDs would behave in a relational database. You have been given the task to make sure that the IDs output from a monthly Spark task start at the highest value from the previous month.

The spark session and two DataFrames, voter_df_march and voter_df_april, are available in your workspace. The pyspark.sql.functions library is available under the alias F.

- Determine the highest ROW_ID in voter_df_march and save it in the variable previous_max_ID. The statement .rdd.max()[0] will get the maximum ID.
- Add a ROW_ID column to voter_df_april starting at the value of previous_max_ID.
- Show the ROW_ID's from both Data Frames and compare.

In [ ]:
# Determine the highest ROW_ID and save it in previous_max_ID
previous_max_ID = voter_df_march.select('ROW_ID').rdd.max()[0]

# Add a ROW_ID column to voter_df_april starting at the desired value
voter_df_april = voter_df_april.withColumn('ROW_ID', F.monotonically_increasing_id() + previous_max_ID)

# Show the ROW_ID from both DataFrames and compare
voter_df_march.select('ROW_ID').show()
voter_df_april.select('ROW_ID').show()

```
<script.py> output:
    +-------------+
    |       ROW_ID|
    +-------------+
    |   8589934592|
    |  25769803776|
    |  34359738368|
    |  42949672960|
    |  51539607552|
    | 103079215104|
    | 111669149696|
    | 231928233984|
    | 240518168576|
    | 360777252864|
    | 395136991232|
    | 601295421440|
    | 635655159808|
    | 670014898176|
    | 807453851648|
    | 850403524608|
    | 944892805120|
    | 962072674304|
    |1005022347264|
    |1047972020224|
    +-------------+
    only showing top 20 rows
    
    +-------------+
    |       ROW_ID|
    +-------------+
    |1717986918400|
    |1735166787584|
    |1743756722176|
    |1752346656768|
    |1760936591360|
    |1812476198912|
    |1821066133504|
    |1941325217792|
    |1949915152384|
    |2070174236672|
    |2104533975040|
    |2310692405248|
    |2345052143616|
    |2379411881984|
    |2516850835456|
    |2559800508416|
    |2654289788928|
    |2671469658112|
    |2714419331072|
    |2757369004032|
    +-------------+
    only showing top 20 rows
```
It's easy to forget that the output of a Spark method can often be modified before being assigned. This provides a lot of power and flexibility, especially when trying to migrate tasks from various technologies. Consider how you could use everything we've learned in this chapter to create a combination ID containing a name, a new ID, and perhaps a conditional value. When you are able to view your tasks as compositions of available functions, you can clean and modify your data in any way you see fit.

---
<a id='perf'></a>

## Improving Performance

<img src="images/spark3_032.png" alt="" style="width: 800px;"/>

<img src="images/spark3_033.png" alt="" style="width: 800px;"/>

<img src="images/spark3_034.png" alt="" style="width: 800px;"/>

<img src="images/spark3_035.png" alt="" style="width: 800px;"/>

<img src="images/spark3_036.png" alt="" style="width: 800px;"/>

## 

In [ ]:
<img src="images/spark3_037.png" alt="" style="width: 800px;"/>

In [ ]:
---
<a id='intro'></a>